In [2]:
from geotext import GeoText
import pandas as pd
import numpy as np
import ast
import csv
import json
import re
import urllib2

In [3]:
def load_json_multiple(segments):
    #chunk = ""
    for segment in segments:
        #chunk += segment
        try:
            yield json.loads(segment)
            #chunk = ""
        except ValueError:
            pass


tweets_json = []

with open('merged.json') as f:
    for parsed_json in load_json_multiple(f):
        tweets_json.append(parsed_json)

In [4]:
text = []
locations = []
for line in tweets_json:
    if 'text' in line:
        text.append(line['text'])
    if 'user' in line:
        if 'location' in line['user']:
            locations.append(line['user']['location'])

In [4]:
regexV = r'.*vaccine.*'
regexO = r'.*outbreak.*'

vaccine = []
outbreak = []
vaccineLoc = []
outbreakLoc = []

for line in text:
    if re.match(regexV, line.encode('utf-8'), re.IGNORECASE):
        vaccine.append(line.encode('utf-8'))
        temp = locations[text.index(line)]
        if temp != None:
            vaccineLoc.append(temp.encode('utf-8'))
        else:
            vaccineLoc.append(temp)
    if re.match(regexO, line.encode('utf-8'), re.IGNORECASE):
        outbreak.append(line.encode('utf-8'))
        temp = locations[text.index(line)]
        if temp != None:
            outbreakLoc.append(temp.encode('utf-8'))
        else:
            vaccineLoc.append(temp)

In [19]:
from itertools import izip
with open('vac_loc.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(izip(vaccine, vaccineLoc))

In [20]:
with open('break_loc.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(izip(outbreak, outbreakLoc))

In [7]:
Lon = []
Lat = []
empty = []
repeat = []

In [8]:
for line in outbreak:
    removeHashTag = line.replace("#", "")
    temp = GeoText(removeHashTag)
    if temp.cities or temp.countries:
        if temp.cities:
            for c in temp.cities:
                tmp = c.replace(" ","%20")
                response = urllib2.urlopen("http://nominatim.openstreetmap.org/search?q="+ tmp +"&format=json&addressdetails=1&limit=1")
                data = json.load(response)
                if len(data) != 0:
                    if (data["lat"] and data["lon"]):
                        tmplat = data[0]['lat'].encode('utf-8')
                        tmplon = data[0]['lon'].encode('utf-8')
                        if tmplat in Lat and tmplon in Lon:
                            repeat[Lat.index(tmplat)] += 1
                        else:
                            Lat.append(tmplat)
                            Lon.append(tmplon)
                            repeat.append(1)
                    else:
                        empty.append(line)
                else:
                    empty.append(c)
        else:
            for co in temp.countries:
                tmp = co.replace(" ","%20")
                response = urllib2.urlopen("http://nominatim.openstreetmap.org/search?q="+ tmp +"&format=json&addressdetails=1&limit=1")
                data = json.load(response)
                if len(data) != 0:
#                     print("country\n")
#                     print('lat' in data[0])
#                     print("\n")
                    if (data[0]['lat'] and data[0]['lon']):
                        tmplat = data[0]['lat'].encode('utf-8')
                        tmplon = data[0]['lon'].encode('utf-8')
                        if tmplat in Lat and tmplon in Lon:
                            repeat[Lat.index(tmplat)] += 1
                        else:
                            Lat.append(tmplat)
                            Lon.append(tmplon)
                            repeat.append(1)
                    else:
                        empty.append(line)
                else:
                    empty.append(co)

In [9]:
with open('break_loc_lon_lat_repeat.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(izip(Lat, Lon, repeat))

In [10]:
vacTextLocation = []
for line in vaccine:
    temp = GeoText(line)
    if temp.cities or temp.countries:
        if temp.cities:
            for c in temp.cities:
                vacTextLocation.append(c)
        else:
            for c in temp.countries:
                vacTextLocation.append(c)
len(vacTextLocation)

604

In [11]:
Lon = []
Lat = []
empty = []

In [12]:
from collections import Counter
temp = Counter(vacTextLocation)
for line in temp:
    repeated = temp[line]
    tmp = line.replace(" ","%20")
    response = urllib2.urlopen("http://nominatim.openstreetmap.org/search?q="+ tmp +"&format=json&addressdetails=1&limit=1")
    data = json.load(response)
    if len(data) != 0:
        if (data[0]['lat'] and data[0]['lon']):
            tmplat = data[0]['lat'].encode('utf-8')
            tmplon = data[0]['lon'].encode('utf-8')
            for i in range(0, repeated):
                Lat.append(tmplat)
                Lon.append(tmplon)             
        else:
            empty.append(line)
    else:
        empty.append(line)

In [13]:
with open('vac_text_lon_lat.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(izip(Lat, Lon))

In [14]:
Lon = []
Lat = []
empty = []
repeat = []

In [15]:
vaccineLocWithoutNone = []

for lc in vaccineLoc:
    if lc != None:
        removeHashTag = lc.replace("#", "")
        vaccineLocWithoutNone.append(removeHashTag)

tmpVacLoc = Counter(vaccineLocWithoutNone)

In [16]:
regexHashtag = r'#(\w+)'

for line in tmpVacLoc:
    if re.match(regexHashtag, line):
        empty.append(line)
        continue
    repeated = tmpVacLoc[line]
    tmp = line.replace(" ","%20")
    response = urllib2.urlopen("http://nominatim.openstreetmap.org/search?q="+ tmp +"&format=json&addressdetails=1&limit=1")
    data = json.load(response)
    if len(data) != 0:
        if (data[0]['lat'] and data[0]['lon']):
            tmplat = data[0]['lat'].encode('utf-8')
            tmplon = data[0]['lon'].encode('utf-8')
            for i in range(0, repeated):
                Lat.append(tmplat)
                Lon.append(tmplon)             
        else:
            empty.append(line)
    else:
        empty.append(line)

In [18]:
with open('vac_loc_lon_lat.csv', 'wb') as f:
    writer = csv.writer(f)
    writer.writerows(izip(Lat, Lon))

In [5]:
len(text)

14018

In [6]:
regexX = r'.*malaria.*day.*'
regexY = r'.*end.*malaria.*'

In [7]:
check = []
for line in text:
    if re.match(regexX, line.encode('utf-8'), re.IGNORECASE) or re.match(regexY, line.encode('utf-8'), re.IGNORECASE):
        check.append(line.encode('utf-8'))

In [10]:
check

['RT @duffyonpoint: #Malaria Remains the highest killer in #Africa! Today, in #Kebbi #Nigeria, We add our voice in this Noble Global fi\xe2\x80\xa6 ',
 "RT @WHOMaldives: It's #WorldMalariaDay2017 Maldives in 2015 was the 1st country in @WHOSEARO to be certified Malaria-free! Read more\xe2\x80\xa6 ",
 "RT @Mapbox: Over 1.7M buildings mapped to fight malaria by #WorldMalariaDay. We're not done yet, keep it going!\xe2\x80\xa6 ",
 "RT @Mapbox: Over 1.7M buildings mapped to fight malaria by #WorldMalariaDay. We're not done yet, keep it going!\xe2\x80\xa6 ",
 "RT @Mapbox: Over 1.7M buildings mapped to fight malaria by #WorldMalariaDay. We're not done yet, keep it going!\xe2\x80\xa6 ",
 'RT @IshantSunaria: On #WorldMalariaDay i pray to lord @Gurmeetramrahim ji for wellbeing of all humans.  \nLord Ji Please protect from #malar\xe2\x80\xa6',
 'MNM: Malaria is not yet defeated - An International Organisation, Malaria No More (MNM), on Wednesday called on c... https://t.co/mHelNccjPn',
 'RT @heal